In [1]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.set_printoptions(precision=4, suppress=True)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## 11.5 기간과 기간 연산
며칠, 몇 개월, 몇 분기, 몇 해 같은 기간은 `Period` 클래스로 표현할 수 있다.

![Period](https://user-images.githubusercontent.com/16831323/130020347-4a7ae42a-2fee-422b-a9ba-77e2ad3824bc.png)

In [28]:
# 2007년 1월 1일부터 같은 해 12월 31일까지의 기간
p = pd.Period(2007, freq='A-DEC')
p
p.start_time, p.end_time
# ✨ remind ) A-DEC : DEC(12월)의 마지막 일을 가리키는 연간 주기

Period('2007', 'A-DEC')

(Timestamp('2007-01-01 00:00:00'), Timestamp('2007-12-31 23:59:59.999999999'))

Period 객체는 정수를 더하거나 빼서 편리하게 정해진 빈도에 따라 기간을 이동시킬 수 있다.

In [29]:
p1 = p + 5
p1
p1.start_time
p1.end_time

Period('2012', 'A-DEC')

Timestamp('2012-01-01 00:00:00')

Timestamp('2012-12-31 23:59:59.999999999')

In [30]:
p2 = p - 2
p2
p2.start_time
p2.end_time

Period('2005', 'A-DEC')

Timestamp('2005-01-01 00:00:00')

Timestamp('2005-12-31 23:59:59.999999999')

두 기간이 같은 빈도를 가질 경우, 두 기간의 차는 둘 사이의 간격된다.

In [5]:
# 2014 - 2007 = 7 year
pd.Period('2014', freq='A-DEC') - p

<7 * YearEnds: month=12>

In [6]:
# 빈도가 다른 경우 에러가 발생한다
pd.Period('2014', freq='A-JUN') - p

IncompatibleFrequency: Input has different freq=A-DEC from Period(freq=A-JUN)

#### PeriodIndex
일반적인 기간 범위는 `period_range` 함수로 생성할 수 있다.<br>
`PeriodIndex` 클래스는 <u>순차적인 기간</u>으로 저장되며 다른 pandas 자료구조에서 축 색인과 마찬가지로 사용될 수 있다.

![period_range](https://i.ibb.co/61hBZqQ/image.png)

In [33]:
rng = pd.period_range('2000-01-01', '2000-06-30', freq='M')
rng

PeriodIndex(['2000-01', '2000-02', '2000-03', '2000-04', '2000-05', '2000-06'], dtype='period[M]', freq='M')

In [34]:
pd.Series(np.random.randn(6), index=rng)

2000-01   -2.001637
2000-02   -0.371843
2000-03    1.669025
2000-04   -0.438570
2000-05   -0.539741
2000-06    0.476985
Freq: M, dtype: float64

문자열 배열을 이용하여 `PeriodIndex` 클래스를 생성하는 것도 가능하다.

In [35]:
values = ['2001Q3', '2002Q2', '2003Q1']
index = pd.PeriodIndex(values, freq='Q-DEC')
index

PeriodIndex(['2001Q3', '2002Q2', '2003Q1'], dtype='period[Q-DEC]', freq='Q-DEC')

### 11.5.1 Period의 빈도 변환
기간와 `PeriodIndex` 객체는 `asfreq` 메서드를 통해 다른 빈도로 변환할 수 있다.

✨ 새해 첫날부터 시작하는 연간 빈도를 월간 빈도로 변환해보자!

In [36]:
p = pd.Period('2007', freq='A-DEC')
p
p.start_time
p.end_time

Period('2007', 'A-DEC')

Timestamp('2007-01-01 00:00:00')

Timestamp('2007-12-31 23:59:59.999999999')

![image](https://user-images.githubusercontent.com/16831323/130014602-8872d012-41f0-49b8-a0bf-309f4df1e9fb.png)


이때, `Period('2007', freq='A-DEC')`는 전체 기간에 대한 커서로 생각할 수 있다! 그리고 이것을 다시 월간으로 나눌 수 있다.

![image](https://user-images.githubusercontent.com/16831323/130015788-c82e639f-e307-4904-ae7a-a07468da06af.png)

In [37]:
p.asfreq('M', how='start')
p.asfreq('M', how='start').start_time
p.asfreq('M', how='start').end_time

Period('2007-01', 'M')

Timestamp('2007-01-01 00:00:00')

Timestamp('2007-01-31 23:59:59.999999999')

In [38]:
p.asfreq('M', how='end')
p.asfreq('M', how='end').start_time
p.asfreq('M', how='end').end_time

Period('2007-12', 'M')

Timestamp('2007-12-01 00:00:00')

Timestamp('2007-12-31 23:59:59.999999999')

빈도가 상위단계에서 하위 단계로 변화되는 경우 상위 기간은 하위 기간이 어디에 속했는지에 따라 결정된다.

❓❓❓❓❓ 상위단계, 하위단계가 무엇일까요? 상위단계가 A-DEC 같은 연간 단위, 하위 단계가 M/W/D 같은 좁은 범위의 시간이라고 생각했는데 코드 상에는 반대인 것 같네요 ㅠㅠ

In [39]:
# 2007년 8월
p = pd.Period('Aug-2007', 'M')
print(f"p=Period('Aug-2007', 'M') : {p.__repr__()}\nstart : {p.start_time}\nend : {p.end_time}\n")

# A-JUN 빈도일 경우 2007년 8월은 실제로 2008년 기간에 속하게 된다.
m_to_a_jun = p.asfreq('A-JUN')
print(f"p.asfreq('A-JUN') : {m_to_a_jun.__repr__()}\nstart : {m_to_a_jun.start_time}\nend : {m_to_a_jun.end_time}\n")

a_jun = pd.Period('2008', 'A-JUN')
print(f"Period('2008', 'A-JUN') : {a_jun.__repr__()}\nstart : {a_jun.start_time}\nend : {a_jun.end_time}")

p=Period('Aug-2007', 'M') : Period('2007-08', 'M')
start : 2007-08-01 00:00:00
end : 2007-08-31 23:59:59.999999999

p.asfreq('A-JUN') : Period('2008', 'A-JUN')
start : 2007-07-01 00:00:00
end : 2008-06-30 23:59:59.999999999

Period('2008', 'A-JUN') : Period('2008', 'A-JUN')
start : 2007-07-01 00:00:00
end : 2008-06-30 23:59:59.999999999


모든 `PeriodIndex` 객체나 시계열은 지금까지 살펴본 내용과 같은 방식으로 변환할 수 있다.

In [40]:
rng = pd.period_range('2006', '2009', freq='A-DEC')
ts = pd.Series(np.random.randn(len(rng)), index=rng)
ts

for i in ts.index:
    print(f'{i.start_time} ~ {i.end_time}')

2006    3.248944
2007   -1.021228
2008   -0.577087
2009    0.124121
Freq: A-DEC, dtype: float64

2006-01-01 00:00:00 ~ 2006-12-31 23:59:59.999999999
2007-01-01 00:00:00 ~ 2007-12-31 23:59:59.999999999
2008-01-01 00:00:00 ~ 2008-12-31 23:59:59.999999999
2009-01-01 00:00:00 ~ 2009-12-31 23:59:59.999999999


In [41]:
# 연 빈도 -> 해당 빈도의 시작 월부터 시작하는 월 빈도
ts_m = ts.asfreq('M', how='start')
ts_m

for i in ts_m.index:
    print(f'{i.start_time} ~ {i.end_time}')

2006-01    3.248944
2007-01   -1.021228
2008-01   -0.577087
2009-01    0.124121
Freq: M, dtype: float64

2006-01-01 00:00:00 ~ 2006-01-31 23:59:59.999999999
2007-01-01 00:00:00 ~ 2007-01-31 23:59:59.999999999
2008-01-01 00:00:00 ~ 2008-01-31 23:59:59.999999999
2009-01-01 00:00:00 ~ 2009-01-31 23:59:59.999999999


In [42]:
# 연 빈도 -> 매 해 마지막 영업일
ts_b = ts.asfreq('B', how='end')
ts_b

for i in ts_b.index:
    print(f'{i.start_time} ~ {i.end_time}')

2006-12-29    3.248944
2007-12-31   -1.021228
2008-12-31   -0.577087
2009-12-31    0.124121
Freq: B, dtype: float64

2006-12-29 00:00:00 ~ 2006-12-29 23:59:59.999999999
2007-12-31 00:00:00 ~ 2007-12-31 23:59:59.999999999
2008-12-31 00:00:00 ~ 2008-12-31 23:59:59.999999999
2009-12-31 00:00:00 ~ 2009-12-31 23:59:59.999999999


### 11.5.2 분기 빈도
- 분기 데이터는 재정/금융 및 다른 분야에서 표준으로 사용된다.
- 많은 분기 데이터는 일반적으로 **회계연도의 끝**인 12월의 마지막 날이나 마지막 업무일을 기준으로 보고한다.
    - `2014Q4`는 회계연도의 끝이 어디인가에 따라 의미가 달라진다.
    - pandas는 12가지(1월~12월) 모든 경우의 수를 지원한다. 👉 Q-JAN, Q-FAB, ... , Q-JAN
    - ![image](https://user-images.githubusercontent.com/16831323/130054596-f5ba40eb-2492-48c8-ab72-5e85d38a1df9.png)


In [43]:
p = pd.Period('2012Q4', freq='Q-JAN')
p
p.asfreq('D', 'start')
p.asfreq('D', 'end')
# p의 시작일은 2011-11-01, 말일은 2012-01-31

Period('2012Q4', 'Q-JAN')

Period('2011-11-01', 'D')

Period('2012-01-31', 'D')

- 기간 연산 예시
    - 분기 영업마감일의 오후 4시를 가리키는 타임스탬프를 구해보자
    - ![image](https://user-images.githubusercontent.com/16831323/130056594-2fce977e-65e0-46fa-af01-9fecbcea05f9.png)

In [44]:
p4pm = (p.asfreq('B', 'e') - 1).asfreq('T', 's') + 16 * 60
# p.asfreq('B', 'e') : 분기 영업 마감일 - 1, 종료 날짜 기준 ⇒ 2012-01-31 00:00:00 ~ 2012-01-31 23:59:59
# asfreq('T', 's') : 주기변경 ⇒ 분기 → 분, 시작 시간 기준 ⇒   2012-01-31 00:00:00 ~ 2012-01-31 00:00:59
# 16 * 60 : 60분 * 16시간 ⇒ 오후 4시 ⇒                        2012-01-30 16:00:00 ~ 2012-01-30 16:00:59
p4pm
p4pm.to_timestamp()

Period('2012-01-30 16:00', 'T')

Timestamp('2012-01-30 16:00:00')

- `period_range`를 사용한 분기 범위

In [45]:
rng = pd.period_range('2011Q3', '2012Q4', freq='Q-JAN')
ts = pd.Series(np.arange(len(rng)), index=rng)
ts

# 연산도 위와 동일한 방법으로 수행할 수 있다.
new_rng = (rng.asfreq('B', 'e') - 1).asfreq('T', 's') + 16 * 60
ts.index = new_rng.to_timestamp()
ts

2011Q3    0
2011Q4    1
2012Q1    2
2012Q2    3
2012Q3    4
2012Q4    5
Freq: Q-JAN, dtype: int32

2010-10-28 16:00:00    0
2011-01-28 16:00:00    1
2011-04-28 16:00:00    2
2011-07-28 16:00:00    3
2011-10-28 16:00:00    4
2012-01-30 16:00:00    5
dtype: int32

### 11.5.3 타임스탬프와 기간 서로 변환하기
`timestamp`로 색인된 Series와 DataFrame 객체는 `to_period` 메서드를 사용해서 기간(period)로 변환 가능하다. 
- 이때, 기간은 <u>겹치지 않는 시간상의 간격</u>이다 👉 주어진 빈도에서 타임스탬프는 하나의 기간에만 속한다.
- 타임스탬프로부터 생성된 `PeriodIndex`는 타임스탬프 값을 통해 **추론**되고 직접 빈도를 지정할수도 있다.

In [46]:
# 타임스탬프 생성
rng = pd.date_range('2000-01-01', periods=3, freq='M')
ts = pd.Series(np.random.randn(3), index=rng)
ts

2000-01-31    0.302614
2000-02-29    0.523772
2000-03-31    0.000940
Freq: M, dtype: float64

In [47]:
type(ts.index[0])

pandas._libs.tslibs.timestamps.Timestamp

In [48]:
# 타임스탬프 → period
pts = ts.to_period()
pts # 추론된 period
type(pts.index[0])

2000-01    0.302614
2000-02    0.523772
2000-03    0.000940
Freq: M, dtype: float64

pandas._libs.tslibs.period.Period

In [49]:
rng = pd.date_range('1/29/2000', periods=6, freq='D')
ts2 = pd.Series(np.random.randn(6), index=rng)
ts2 # 직접 빈도 지정
ts2.to_period('M')

2000-01-29    1.343810
2000-01-30   -0.713544
2000-01-31   -0.831154
2000-02-01   -2.370232
2000-02-02   -1.860761
2000-02-03   -0.860757
Freq: D, dtype: float64

2000-01    1.343810
2000-01   -0.713544
2000-01   -0.831154
2000-02   -2.370232
2000-02   -1.860761
2000-02   -0.860757
Freq: M, dtype: float64

- period to timestamp : `to_timestamp`

In [50]:
pts = ts2.to_period()
pts
pts.to_timestamp(how='end')

2000-01-29    1.343810
2000-01-30   -0.713544
2000-01-31   -0.831154
2000-02-01   -2.370232
2000-02-02   -1.860761
2000-02-03   -0.860757
Freq: D, dtype: float64

2000-01-29 23:59:59.999999999    1.343810
2000-01-30 23:59:59.999999999   -0.713544
2000-01-31 23:59:59.999999999   -0.831154
2000-02-01 23:59:59.999999999   -2.370232
2000-02-02 23:59:59.999999999   -1.860761
2000-02-03 23:59:59.999999999   -0.860757
Freq: D, dtype: float64

### 11.5.4 배열로 PeriodIndex 생성하기
고정 빈도를 가지는 데이터는 종종 여러 컬럼에 걸쳐 기간에 대한 정보를 함께 저장하기도 한다.

In [51]:
# 연도와 분기가 구분되 컬럼이 존재
# year, quarter
data = pd.read_csv('examples/macrodata.csv')
data.head(5)

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [52]:
data.year

0      1959.0
1      1959.0
2      1959.0
3      1959.0
4      1960.0
        ...  
198    2008.0
199    2008.0
200    2009.0
201    2009.0
202    2009.0
Name: year, Length: 203, dtype: float64

In [53]:
data.quarter

0      1.0
1      2.0
2      3.0
3      4.0
4      1.0
      ... 
198    3.0
199    4.0
200    1.0
201    2.0
202    3.0
Name: quarter, Length: 203, dtype: float64

- `PeriodIndex`에 빈도값과 함께 위의 컬럼을 전달하면 DataFrame에서 사용할 수 있는 색인을 만들어 낼 수 있다.

In [54]:
index = pd.PeriodIndex(year=data.year, quarter=data.quarter,
                       freq='Q-DEC')
index

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', length=203, freq='Q-DEC')

In [55]:
data.index = index
data

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
1959Q1,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.980,139.7,2.82,5.8,177.146,0.00,0.00
1959Q2,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.150,141.7,3.08,5.1,177.830,2.34,0.74
1959Q3,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.350,140.5,3.82,5.3,178.657,2.74,1.09
1959Q4,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.370,140.0,4.33,5.6,179.386,0.27,4.06
1960Q1,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.540,139.6,3.50,5.2,180.007,2.31,1.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008Q3,2008.0,3.0,13324.600,9267.7,1990.693,991.551,9838.3,216.889,1474.7,1.17,6.0,305.270,-3.16,4.33
2008Q4,2008.0,4.0,13141.920,9195.3,1857.661,1007.273,9920.4,212.174,1576.5,0.12,6.9,305.952,-8.79,8.91
2009Q1,2009.0,1.0,12925.410,9209.2,1558.494,996.287,9926.4,212.671,1592.8,0.22,8.1,306.547,0.94,-0.71
2009Q2,2009.0,2.0,12901.504,9189.0,1456.678,1023.528,10077.5,214.469,1653.6,0.18,9.2,307.226,3.37,-3.19
